In [22]:
import pyspark
from src.etl import *
from src.utils import *
import yaml
from collections import Counter
from collections import defaultdict

# once intialized, it's global in the jupyter notebook kernal; one context per notebook
sc = pyspark.SparkContext.getOrCreate('local[*]')

AttributeError: 'str' object has no attribute 'get'

In [ ]:
big_list = range(10000)
rdd = sc.parallelize(big_list, 2)
odds = rdd.filter(lambda x: x % 2 != 0)
odds.take(5)

In [18]:
rdd

NameError: name 'rdd' is not defined

In [17]:
"""
SELECT subject_id, sex, dob, ifnull(dod, "")
FROM PATIENT
"""

SyntaxError: EOL while scanning string literal (<ipython-input-17-445376adb8eb>, line 1)

## meta

In [ ]:
from src.etl import *
from src.utils import *
import yaml
from collections import Counter
from collections import defaultdict

In [ ]:
params['data']['image_size']

In [ ]:
META = etl()

In [ ]:
print('label distribution: ', Counter(META.label), sep='\n')

## train / test split

In [ ]:
META = dataset_split(META)
META.head()

In [ ]:
print('label integer map: ', params['train']['labelmap'], sep='\n', end='\n\n')
for ds in (0, 1):
    print('{0} data from covid datasets: '.format({0: 'test', 1: 'train'}[ds]), Counter(META[META.train==ds].label), sep='\n', end='\n\n')

In [ ]:
# cache image meta
if not os.path.isdir(SAVE_PATH):
    os.makedirs(SAVE_PATH)
with open(os.path.join(SAVE_PATH, 'meta'), 'wb') as pickle_file:
    pickle.dump(META, pickle_file, pickle.HIGHEST_PROTOCOL)
    print(f"META data saved at {os.path.join(SAVE_PATH, 'meta')}")

In [ ]:
%%time
if False:
    # -------------------------------------------------------------------------------------------------------------------
    # imagge file to matrix  
    # NOTE: this needs model to load all data at once into cache which take up too much storage, 
    #       making computing inefficient, NOT GOOD
    # -------------------------------------------------------------------------------------------------------------------
    # train_data / test_data structure: 
    #                        {'covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer},
    #                        '!covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer}}
    TRAIN_DATA, TEST_DATA = datafromfile(META)
    # a sanity check
    print(f"train covid sample #: {len(TRAIN_DATA['covid']['label'])}")
    print(f"train !covid sample #: {len(TRAIN_DATA['!covid']['label'])}")
    print(f"test  covid sample #: {len(TEST_DATA['covid']['label'])}")
    print(f"test  !covid sample #: {len(TEST_DATA['!covid']['label'])}")
    # cache image data
    with open(os.path.join(SAVE_PATH, 'train.data'), 'wb') as pickle_file:
        pickle.dump(TRAIN_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'train.data')}")
    with open(os.path.join(SAVE_PATH, 'test.data'), 'wb') as pickle_file:
        pickle.dump(TEST_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'test.data')}")
else:
    # merge imgaes together instead, and transform image to batches of matrix during training
    mergesoure(META)

In [ ]:
# sanity check for image shape, they need to consistent with model input
import cv2
import glob
import numpy as np
for i in np.random.choice(META.index, 10):
    print(i, cv2.imread(META.loc[i, 'imgid']).shape)